# Tutorial: How to use arbocartoR
This tutorial introduces the **arbocartoR** package, which simulates:
- Spatial and temporal dynamics of *Aedes* mosquito populations under natural and controlled conditions.
- Epidemiological dynamics of arboviruses (*dengue, zika, chikungunya*) across human and vector populations.

## Setup
Install arbocartoR package:

In [ ]:
# remotes::install_gitlab("astre/arbocartoR", host = "https://gitlab.cirad.fr")

Load required packages:

In [ ]:

suppressPackageStartupMessages({
  suppressWarnings({
    library(arbocartoR)
    library(data.table)
    library(magrittr)
    library(tidyterra)
    library(ggplot2)
    library(sf)
    library(TDLM)
  })
})


## General Data
The package requires two main datasets:
- **parcels**: each row is a node/parcel with attributes (ID: parcel unique identifier, POP: resident population, Kfix: fix/anthropogenic carrying capacity of the parcel, Kvar: variable/rainfall-dependant carying capacity, etc.).
- **meteo**: meteorological data (ID, DATE, rainfall `RR` - in mm, temperature `TP` - in degrees).

The structure and attributes of the data are detailled in the helpers.

In [ ]:
# ? parcels

In [ ]:
# ? meteo

An example for both datasets is provided in the package:

In [ ]:
data(parcels)
data(meteo)

head(parcels)
head(meteo)

### Select a subset of parcels
Lets use only 4 parcels to run quick simulations.

In [ ]:

parcels <- parcels[startsWith(ID, "06") & (grepl("Gatt", NOM_COM) | grepl("Saint-Jeannet", NOM_COM) | grepl("La Gaude", NOM_COM)), ]


### Run a simple simulation
Select the species of interest, inform the function with your parcels and meteo objects, as well as the temporal boundaries of the simulation. The period must be covered by the meteorological dataset and must last at least a year (for initialization purposes).
Note that the vector population dynamics is deterministic, running multiple simulations would produce identical outputs.

In [ ]:
traj <- run_arbocartoR(
  vector = "Ae. aegypti",
  parcels = parcels,
  meteo = meteo,
  n_sim = 1,
  start_date = min(meteo$DATE) %>% as.Date,
  end_date = max(meteo$DATE) %>% as.Date
)

Explore the simulated trajectories compiled as a list of data.table with one data.table per simulation. Use `plot_TS` function to visualize the trajectiories.

In [ ]:
head(traj[[1]])

plot_TS(traj, stage = "newEggs")

### Explore and update parameters
Use the `build_gdata` function to build the set of ecological and epidemiological parameters required by the model. 
Explore the default parameters for *aegypti*.

In [ ]:
# ?build_gdata()
gdata_default <- build_gdata(vector = "Ae. aegypti")

Update the parameters for *aegypti*.

In [ ]:
#Change parameters
gdata_updated <- build_gdata(vector = "Ae. aegypti",
            bMH = 0.5,
            muE = 0.001)

Compare them:

In [ ]:
gdata_default[c("bMH","muE","mu1L")]

gdata_updated[c("bMH","muE","mu1L")]

Run a simulation with those data

In [ ]:
traj <- run_arbocartoR(
  vector = "Ae. aegypti",
  parcels = parcels,
  meteo = meteo,
  gdata = gdata_updated,
  n_sim = 1,
  start_date = min(meteo$DATE) %>% as.Date,
  end_date = max(meteo$DATE) %>% as.Date
)

head(traj[[1]])

plot_TS(traj, stage = "newEggs")

## Local Preventive Control
Initialize dataset for preventive control actions.
Preventive control measures should be described in a data.frame or data.table using the mandatory following columns: 'action', 'loc', 'start', 'end', 'p'.

### Define control actions

In [ ]:
prev_control <- data.table(
  action = character(),
  loc = factor(),
  start = structure(numeric(0), class = "Date"),
  end = structure(numeric(0), class = "Date"),
  p = numeric()
)

# Reduce carrying capacity (breeding sites destruction)
prev_control <- rbindlist(list(prev_control, data.table(
  action = "K",
  loc = "060640000",
  start = as.Date("2022/07/01"),
  end = as.Date("2022/07/07"),
  p = 0.2
)), fill = TRUE)

# Increase larvae mortality (larviciding)
prev_control <- rbindlist(list(prev_control, data.table(
  action = "L",
  loc = "060640000",
  start = as.Date("2022/07/01"),
  end = as.Date("2022/07/07"),
  p = 0.2
)), fill = TRUE)

prev_control <- rbindlist(list(prev_control, data.table(
  action = "L",
  loc = "061220000",
  start = as.Date("2022/07/07"),
  end = as.Date("2022/07/15"),
  p = 0.5
)), fill = TRUE)

# Increase adult mortality (fumigation)
prev_control <- rbindlist(list(prev_control, data.table(
  action = "A",
  loc = "061220000",
  start = as.Date("2022/07/01"),
  end = as.Date("2022/07/07"),
  p = 0.2
)), fill = TRUE)

prev_control <- rbindlist(list(prev_control, data.table(
  action = "A",
  loc = "060640000",
  start = as.Date("2022/07/01"),
  end = as.Date("2022/07/07"),
  p = 0.8
)), fill = TRUE)


You can also use `build_prev_control()` function to format it using GPS coordinates and buffer width (area covered by the control activities). 
A SpatVector object is required to calculate the spatial intersection between the parcels and the area covered by the control activities.

In [ ]:
SpatVec <- system.file("shape/SpatVec.shp", package = "arbocartoR") %>% terra::vect(.)
SpatVec <- SpatVec[SpatVec$ID %in% parcels$ID, ]

prev_control_func <- build_prev_control(action = c("K", "L"),
                   lon = 1034406, lat = 6306132,
                   start = as.Date("2022/07/01"),
                   end = as.Date("2022/07/07"),
                   p = c(0.2, 0.8),
                   SpatVec = SpatVec,
                   buffer_width = 100)

### Run simulation with preventive control

In [ ]:
traj <- run_arbocartoR(
  vector = "Ae. aegypti",
  parcels = parcels,
  meteo = meteo,
  n_sim = 1,
  start_date = min(meteo$DATE) %>% as.Date,
  end_date = max(meteo$DATE) %>% as.Date,
  prev_control = prev_control
)

prev_control
plot_TS(traj, stage = "Lm")

## Introduction of Exposed Individuals
Define an introduction event modifying the epidemiological state of the population at a given time.

In [ ]:
introduction_pts <- build_E_random(
  period_start = as.Date("2022/07/01"),
  period_end   = as.Date("2022/07/15"),
  n_ind = 10,
  n_events = 1,
  stage = "Eh",
  loc = parcels$ID
)

introduction_pts

traj <- run_arbocartoR(
  vector = "Ae. aegypti",
  parcels = parcels,
  meteo = meteo,
  n_sim = 5,
  start_date = as.Date("2022/01/01"),
  end_date = as.Date("2022/11/30"),
  introduction_pts = introduction_pts,
  initMosq = 1e5
)

plot_TS(traj, stage = c("Eh", "Ih", "Rh"))

## Human Mobility
Introduce human mobility between parcels based on the spatial structure of the area of interest, distribution law based on distance between parcels and resident population.

In [ ]:
SpatVec <- system.file("shape/SpatVec.shp", package = "arbocartoR") %>% terra::vect(.)
SpatVec <- SpatVec[SpatVec$ID %in% parcels$ID, ]

mMov <- build_mMov(SpatVec,
                   law = "NGravExp",
                   p2move = 0.715,
                   verbose = TRUE)

SpatVec$pMovij <- mMov$proba_ij[rownames(mMov$proba_ij) == "060650102",]
SpatVec$pMovji <- mMov$proba_ji[rownames(mMov$proba_ji) == "060650102",]

ggplot(SpatVec) +
  geom_spatvector(aes(fill = pMovij), color = NA) +
  scale_fill_continuous(low = "white", high = "#590003") +
  ggtitle("Probability to visit a parcel (residents of 061220000)") +
  labs(fill = "")

ggplot(SpatVec) +
  geom_spatvector(aes(fill = pMovji), color = NA) +
  scale_fill_continuous(low = "white", high = "#590003") +
  ggtitle("Probability of visiting 061220000 (depending on residence)") +
  labs(fill = "")

#### Run the simulation with mobility

In [ ]:
traj_mMov <- run_arbocartoR(
  vector = "Ae. aegypti",
  parcels = parcels,
  meteo = meteo,
  mMov = mMov,
  n_sim = 1,
  start_date = as.Date("2022/01/01"),
  end_date = as.Date("2022/11/30"),
  introduction_pts = introduction_pts,
  initMosq = 1e5
)

#### Explore and compare the output

In [ ]:
message("Parcels of introduction")
introduction_pts

message("Infected parcels without mobility")
traj[[1]][Eh > 0, unique(ID)] 

message("Infected parcels with mobility")
traj_mMov[[1]][Eh > 0, unique(ID)] 
